In [75]:
import os,glob
import time
import copy
import math
import random
import csv

import numpy as np
import pandas as pd
import seaborn as sns

import imageio
from matplotlib import pyplot as plt
from file_io import load_image

%who

agg	 anno	 annofile	 arange	 brain	 copy	 csv	 detect_edges2d	 df	 
df_brain	 df_signal	 df_signal_new	 get_arangement	 get_brain_outline2d	 get_brains_corr	 get_colorcode16_list	 get_dict	 get_section_array	 
get_section_boundary	 glob	 i	 imageio	 j	 k	 lines	 load_image	 math	 
np	 os	 pd	 plot_brain_regional_density	 plot_brains_signal_diversity	 plot_regional_density	 plot_regional_signal	 plt	 random	 
rotate2d	 rotate3d	 sns	 time	 value	 write_json	 


In [76]:
df_signal = pd.read_csv('../../assets/brain205_signal_region671.csv')
df_signal['n671_u16_id'] = df_signal['n671_u16_id'].astype(int)
df_signal['count'] = df_signal['count'].astype(int)
df_signal['brain'] = df_signal['brain'].astype(str)
df_signal['modality'] = df_signal['modality'].astype(str)
df_signal['label'] = df_signal['label'].astype(str)
df_signal['labelbrain_num'] = df_signal['labelbrain_num'].astype(int)
df_signal

/home/lyf/anaconda3/envs/pytorch1.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,n671_u16_id,count,sizecount,brain,scale,counts,modality,label,labelbrain_num
0,203,328776,29204,191813,11.257927,449668945,U19_Zeng,Plxnd1,20
1,231,93609,8315,191813,11.257927,449668945,U19_Zeng,Plxnd1,20
2,723,32197,2860,191813,11.257927,449668945,U19_Zeng,Plxnd1,20
3,725,34190,3037,191813,11.257927,449668945,U19_Zeng,Plxnd1,20
4,749,8736,776,191813,11.257927,449668945,U19_Zeng,Plxnd1,20
...,...,...,...,...,...,...,...,...,...
194534,463,2991,459,A3_16um_final,6.518387,336020149,Osten,0,2
194535,936,4849,744,A3_16um_final,6.518387,336020149,Osten,0,2
194536,375,1173,180,A3_16um_final,6.518387,336020149,Osten,0,2
194537,1017,12059,1850,A3_16um_final,6.518387,336020149,Osten,0,2


# plot montage to review special brains

In [ ]:
def plot_special_brain(df_signal):
    labels = [l for l in np.unique(df_signal['label']) if l!='0']
    brains = [np.unique(df_signal[df_signal['label']==l]['brain']).tolist() for l in labels]
    braincounts = [[df_signal[df_signal['brain']==b]['counts'].values[0] for b in bs] for bs in brains]
  
    #fig,ax = plt.subplots(2,2)
    im = 0
    ims = 0
    mset = []
    nameset = []
    ifig = 0
    for l,bs,bcs in zip(labels,brains,braincounts):
        bs = sorted(bs,key=lambda b:bcs[bs.index(b)])
        bcs = sorted(bcs)
        #print(l,bs,bcs,'\n')
        
        os.makedirs(f'../brains_review/{l}',exist_ok=True)
        for b,bc in zip(bs,bcs):
            rawfile = f'/PBshare/SEU-ALLEN/Users/ZhixiYun/data/registration/brainimg/CH2_Brains_DS_BE/{b}.v3draw'
            mipfile = f'../brains_review/{l}/{b}.png'
            if os.path.exists(rawfile):
                image = load_image(rawfile)[0]
                axis = image.shape.index(min(image.shape))
                mip = image.max(axis=axis)
                imageio.imsave(mipfile,mip)
                
                im += 1
                ims += 1
                print(ims,ifig,im,b,bc,mip.shape)
                mset.append(mipfile)
                nameset.append(f'{l}_{b}_{bc}')
#                     iax = [0,0,1,1][im-1]
#                     jax = [0,1,0,1][im-1]
#                     ax[iax,jax].imshow(mip,cmap='gray')
#                     ax[iax,jax].axis('off')
#                     ax[iax,jax].text(0.5,0.5,f'{l} {b} {bc}',fontsize=8,color='red',ha='center',va='center')
                if im==20: 
                    ifig += 1 
                    montage_str = f'montage {" ".join(mset)} -tile 4x5 ../brains_review/{ifig}.png'
                    print(montage_str)
                    os.system(montage_str)
#                     plt.subplots_adjust(left=0.01,bottom=0.01,right=0.99,top=0.9,wspace=0.01,hspace=0.1)
#                     plt.savefig(f'{ifig}.png',)
#                     plt.close()
#                     fig,ax = plt.subplots(2,2)
                    im=0 
                    mset = []
                    nameset = []
            
            else:
                print(f'{b} does not exist')
    print(f'\n all brains {ims}')
    if im>0:
            ifig += 1 
            montage_str = f'montage {" ".join(mset)} -tile 4x5 ../brains_review/{ifig}.png'
            print(montage_str)
            os.system(montage_str)

plot_special_brain(df_signal) 

# plot regional signal of some lines

In [77]:
def rotate3d(img3d):
    img = np.zeros((img3d.shape[1],img3d.shape[0],img3d.shape[2]))
    for i in range(img3d.shape[2]):
        img[:,:,i] = img3d[:,:,i].T
    img = img.astype(np.uint8)
    return img

def rotate2d(img2d,rot=90):
    k = int(rot/90)
    img = np.rot90(img2d,k)
    return img

def get_section_array(imagefile,axis=None,rot=0):
    if imagefile.endswith('nrrd'): image = load_image(imagefile)
    else: image = load_image(imagefile)[0]
    if axis==None:
        axis = image.shape.index(min(image.shape))
        if axis==0:
            section = eval(f'image[{int(image.shape[axis]/2)},:,:]')
        elif axis==1:
            section = eval(f'image[:,{int(image.shape[axis]/2)},:]')
        else:
            section = eval(f'image[:,:,{int(image.shape[axis]/2)}]')
    imageio.imsave('test.png',section)
    if rot:
        return rotate2d(section,rot),axis
    else:
        return section,axis
    
def detect_edges2d(img2d):
    img = img2d.astype(float)
    gx, gy = np.gradient(img)
    edges = (gx*gx + gy*gy) != 0
    return edges

def get_brain_outline2d(mask, axis=0, v=255):
    mask = mask > 0
    mask2d = mask.max(axis=axis)
    outline = detect_edges2d(mask2d)
    return outline.astype(np.uint8) * v

def get_section_boundary(array,axis=1,v=255,withoutline=False):
    if len(array.shape)==3:
        if axis==0:
            section = eval(f'array[{int(array.shape[axis]/2)},:,:]')
        elif axis==1:
            section = eval(f'array[:,{int(array.shape[axis]/2)},:]')
        else:
            section = eval(f'array[:,:,{int(array.shape[axis]/2)}]')
        boundary = detect_edges2d(section)
        if withoutline:
            outline = get_brain_outline2d(array, axis=axis, v=v)    
            return np.maximum(boundary.astype(np.uint8) * v  , outline)
    else:
        boundary = detect_edges2d(array)
    return boundary.astype(np.uint8) * v    


    
# section,axis = get_section_array('../../assets/annotation_25.nrrd',None,270)
# boundary = get_section_boundary(section)
# imageio.imsave('../plots/boundary.png',boundary)

In [ ]:
# e = []
# n671_u16_voxel_dict = get_dict(671,16,key='voxel')
# for i in df_signal['n671_u16_id']:
#     if i in n671_u16_voxel_dict.keys():
#         pass#print(i,n671_u16_voxel_dict[i])
#     else: e.append(i)
# elist = np.unique(e).tolist()
# print('in region counts but not in n671_u16: ',elist)
# elist32 = get_u32_from_u16_list(elist)
# print(elist32)
# print(np.unique(elist32).tolist())   

In [81]:
from utils import get_dict

def get_colorcode16_list(value_list,nc=10,cmap='spring'):
    rgblist = (np.array(sns.color_palette(cmap,nc+1))*255).astype(int)
    rgblist = rgblist[nc::-1,:]
    code16_list = ["".join([hex(c)[2:].rjust(2,'0').upper() for c in rgb]) for rgb in rgblist]
    value_code16_list = [code16_list[v] for v in value_list]
    return value_code16_list

def plot_regional_density(df_signal,lines):
    level = 671
    uint = 16
    n671_u16_voxel_dict = get_dict(level,uint,key='voxel')
    voxel_list = [(n671_u16_voxel_dict[i] if i in n671_u16_voxel_dict.keys() else float('inf')) for i in df_signal['n671_u16_id']]
    df_signal['density'] = df_signal['count']/voxel_list
    df_signal['density'] = df_signal['density']/(0.025*0.025*0.025)
    df_signal['logdensity'] = np.log10(df_signal['density']+1).round(2)
    md = df_signal['logdensity'].max()
    nc = 20
    df_signal['ncolor'] = (nc*df_signal['logdensity']/md).astype(int)
    df_signal['colorcode'] = get_colorcode16_list(df_signal['ncolor'],nc=nc)
    labels = [l for l in np.unique(df_signal['label']) if (l!='0' and l in lines)]
    brains = [np.unique(df_signal[df_signal['label']==l]['brain']).tolist() for l in labels]
    braincounts = [[df_signal[df_signal['brain']==b]['counts'].values[0] for b in bs] for bs in brains]
    for l,bs,bcs in zip(labels,brains,braincounts):
        df_label = df_signal[df_signal['label']==l]
        bs = sorted(bs,key=lambda b:bcs[bs.index(b)])
        bcs = sorted(bcs)
        print('\n',l,bs,bcs)  
        fig = plt.figure()
        ib = 0
        for b,bc in zip(bs,bcs):
            df_brain = df_label[df_label['brain']==b]
            #annofile = f'/PBshare/SEU-ALLEN/Users/ZhixiYun/data/registration/Inverse/{b}.v3draw'
            annofile = f'../../assets/n671_u16.nrrd'
            if os.path.exists(annofile):#and os.path.exists(rawfile):
                if len(bs)>7:
                    fig.add_subplot(2,len(bs)/2,len(bs)-ib)
                else:
                    fig.add_subplot(1,len(bs),len(bs)-ib)
                anno,_ = get_section_array(annofile,None,180)
                for region,color in zip(df_brain['n671_u16_id'],df_brain['colorcode']):
                    mask = anno==region
                    indices = np.nonzero(mask)
                    plt.scatter(indices[1], indices[0], s=0.5, c='#'+color, alpha=1.0, edgecolors='none')
                print(f'{b} color mapping done, total voxels {bc}')
                boundary = get_section_boundary(anno)
                indices = np.nonzero(boundary)
                plt.scatter(indices[1], indices[0], s=0.1, c='black', alpha=1.0, edgecolors='none')
                print(f'{b} boundary plotting done')
                plt.axis('off')
                ib += 1
        plt.savefig(f'../plots/{l}_density.png',dpi=300) 
        plt.close()   
    return df_signal
               
def plot_regional_signal(df_signal,lines):
    #templatefile = '../../assets/annotation_25.nrrd'
    #template = get_section_array(templatefile)
    df_signal['logcount'] = np.log(df_signal['count'])
    labels = [l for l in np.unique(df_signal['label']) if (l!='0' and l in lines)]
    maxcount = [df_signal[df_signal['label']==l]['count'].max() for l in labels]
    maxlogcount = [df_signal[df_signal['label']==l]['logcount'].max() for l in labels]
    brains = [np.unique(df_signal[df_signal['label']==l]['brain']).tolist() for l in labels]
    braincounts = [[df_signal[df_signal['brain']==b]['counts'].values[0] for b in bs] for bs in brains]
    nc = 200
    for l,mc,mlc,bs,bcs in zip(labels,maxcount,maxlogcount,brains,braincounts):
        df_label = df_signal[df_signal['label']==l]
        df_label['ncolor'] = (nc*df_label['logcount']/mlc).astype(int)
        df_label['colorcode'] = get_colorcode16_list(df_label['ncolor'],nc=nc)
        bs = sorted(bs,key=lambda b:bcs[bs.index(b)])
        bcs = sorted(bcs)
        print('\n',l,mc,mlc,bs,bcs)  
        fig = plt.figure(figsize=(20,10))        
        ib = 0
        for b,bc in zip(bs,bcs):
            df_brain = df_label[df_label['brain']==b]
            #rawfile = f'/PBshare/SEU-ALLEN/Users/ZhixiYun/data/registration/brainimg/CH2_Brains_DS_BE/{b}.v3draw'
            #annofile = f'/PBshare/SEU-ALLEN/Users/ZhixiYun/data/registration/Inverse/{b}.v3draw'
            annofile = f'../../assets/n671_u16.nrrd'
            if os.path.exists(annofile):#and os.path.exists(rawfile):
                ib += 1
                fig.add_subplot(2,6,ib)
                #raw = get_section_array(rawfile)
                anno,_ = get_section_array(annofile,None,180)
                #fig,ax = plt.subplots(1,2)
                #ax[0].imshow(raw)
                #ax[1].imshow(anno)
                for region,color in zip(df_brain['n671_u16_id'],df_brain['colorcode']):
                    mask = anno==region
                    indices = np.nonzero(mask)
                    plt.scatter(indices[1], indices[0], s=0.5, c='#'+color, alpha=1.0, edgecolors='none')
                print(f'{b} color mapping done, total voxels {bc}')
                boundary = get_section_boundary(anno)
                indices = np.nonzero(boundary)
                plt.scatter(indices[1], indices[0], s=0.1, c='black', alpha=1.0, edgecolors='none')
                print(f'{b} boundary plotting done')
                plt.axis('off')
        plt.savefig(f'../plots/{l}.png',dpi=300) 
        plt.close()

lines = ['Chrna2,Sst','Fezf2','Tlx3','Tnnt1']
#plot_regional_signal(df_signal,lines)
df = plot_regional_density(df_signal,lines)


 Chrna2,Sst ['191798', '191799', '191797'] [2410020130, 2639508389, 2761752975]
191798 color mapping done, total voxels 2410020130
191798 boundary plotting done
191799 color mapping done, total voxels 2639508389
191799 boundary plotting done
191797 color mapping done, total voxels 2761752975
191797 boundary plotting done

 Fezf2 ['18468', '18470', '18469', '18047', '194069', '194071', '211783', '194072', '182724', '211784', '182725', '191807'] [1440, 903264, 4277156, 10917577, 146299120, 331904854, 375605991, 451260912, 513712622, 581307834, 603428441, 646682446]


<ipython-input-81-0ed45a2dd762>:38: MatplotlibDeprecationWarning: Passing non-integers as three-element position specification is deprecated since 3.3 and will be removed two minor releases later.
  fig.add_subplot(2,len(bs)/2,len(bs)-ib)


18468 color mapping done, total voxels 1440
18468 boundary plotting done
18470 color mapping done, total voxels 903264
18470 boundary plotting done
18469 color mapping done, total voxels 4277156
18469 boundary plotting done
18047 color mapping done, total voxels 10917577
18047 boundary plotting done
194069 color mapping done, total voxels 146299120
194069 boundary plotting done
194071 color mapping done, total voxels 331904854
194071 boundary plotting done
211783 color mapping done, total voxels 375605991
211783 boundary plotting done
194072 color mapping done, total voxels 451260912
194072 boundary plotting done
182724 color mapping done, total voxels 513712622
182724 boundary plotting done
211784 color mapping done, total voxels 581307834
211784 boundary plotting done
182725 color mapping done, total voxels 603428441
182725 boundary plotting done
191807 color mapping done, total voxels 646682446
191807 boundary plotting done

 Tlx3 ['17304', '17543', '17544'] [14825488, 366136584, 81

In [71]:
def get_colorcode16_list(value_list,nc=10,cmap='hot_r'):
    rgblist = (np.array(sns.color_palette(cmap,nc+1))*255).astype(int)
    code16_list = ["".join([hex(c)[2:].rjust(2,'0').upper() for c in rgb]) for rgb in rgblist]
    value_code16_list = [code16_list[v] for v in value_list]
    return value_code16_list

def plot_brain_regional_density(df,brain,annofile):
    df_brain = df[df['brain']==brain]
    print(df_brain.shape)
    level = 671
    uint = 16
    n671_u16_voxel_dict = get_dict(level,uint,key='voxel')
    voxel_list = [(n671_u16_voxel_dict[i] if i in n671_u16_voxel_dict.keys() else float('inf')) for i in df_brain[f'n{level}_u{uint}_id']]    
    df_brain['density'] = df_brain['count']/voxel_list
    df_brain['density'] = df_brain['density']/(0.025*0.025*0.025)
    maxd = df_brain['density'].max()
    mind = df_brain['density'].min()
    print(f'{brain} calc density dobe, max density {maxd}, min {mind}')
    df_brain['logdensity'] = np.log10(df_brain['density']+1).round(2)
    nc = 10
    mld = df_brain['logdensity'].max()
    df_brain['ncolor'] = (nc*df_brain['logdensity']/mld).astype(int)
    df_brain['colorcode'] = get_colorcode16_list(df_brain['ncolor'],nc=nc)
    fig = plt.figure()
    if os.path.exists(annofile):#and os.path.exists(rawfile):
        anno,_ = get_section_array(annofile,None,180)
        fig.add_subplot(111)
        for region,color in zip(df_brain['n671_u16_id'],df_brain['colorcode']):
            mask = anno==region
            indices = np.nonzero(mask)
            plt.scatter(indices[1], indices[0], s=0.5, c='#'+color, alpha=1.0, edgecolors='none')
        print(f'{brain} color mapping done')
        boundary = get_section_boundary(anno)
        indices = np.nonzero(boundary)
        plt.scatter(indices[1], indices[0], s=0.1, c='black', alpha=1.0, edgecolors='none')
        print(f'{brain} boundary plotting done')
        plt.axis('off')
    plt.savefig(f'../plots/{brain}_density.png',dpi=300) 
    plt.close()  
    return df_brain
        
        
        
brain='AZ10_SR3B_6_A_647'
#annofile='/PBshare/SEU-ALLEN/Users/ZhixiYun/data/registration/Inverse/WHOLE_mouse_Brain_AZ10_SR3B_6_A_647.v3draw'
annofile='../../assets/n671_u16.nrrd'
df_brain = plot_brain_regional_density(df_signal,brain,annofile)
df_brain

(1160, 13)
AZ10_SR3B_6_A_647 calc density dobe, max density 19473187.77292576, min 0.0


/home/lyf/anaconda3/envs/pytorch1.8/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


AZ10_SR3B_6_A_647 color mapping done
AZ10_SR3B_6_A_647 boundary plotting done


,n671_u16_id,count,sizecount,brain,scale,counts,modality,label,labelbrain_num,density,logdensity,ncolor,colorcode
192580,1912,13948,2623,AZ10_SR3B_6_A_647,5.317953,190729736,Wu,0,2,15107.500677,4.18,5,FF5900
192581,1974,47,9,AZ10_SR3B_6_A_647,5.317953,190729736,Wu,0,2,86.016586,1.94,2,FFFF02
192582,1958,75222,14145,AZ10_SR3B_6_A_647,5.317953,190729736,Wu,0,2,54188.424394,4.73,6,FF2100
192583,1872,1001466,188318,AZ10_SR3B_6_A_647,5.317953,190729736,Wu,0,2,374598.620690,5.57,7,E90000
192584,1910,37,7,AZ10_SR3B_6_A_647,5.317953,190729736,Wu,0,2,44.026327,1.65,2,FFFF02
...,...,...,...,...,...,...,...,...,...,...,...,...,...
193735,1029,1153,217,AZ10_SR3B_6_A_647,5.317953,190729736,Wu,0,2,2524.097828,3.40,4,FF9200
193736,2284,31,6,AZ10_SR3B_6_A_647,5.317953,190729736,Wu,0,2,3426.597582,3.53,4,FF9200
193737,1033,132,25,AZ10_SR3B_6_A_647,5.317953,190729736,Wu,0,2,420.507715,2.62,3,FFC900
193738,391,6094,1146,AZ10_SR3B_6_A_647,5.317953,190729736,Wu,0,2,335065.292096,5.53,7,E90000


In [74]:
for i in range(21):
    j = df['logdensity'][df['ncolor']==i].max()
    k = df['density'][df['ncolor']==i].max()
    print(i,j,k)
print(df['density'].max())

0 0.41 1.5670671053638376
1 0.82 5.665405616951489
2 1.23 16.164677653596005
3 1.64 43.153120019615045
4 2.05 112.49138975321249
5 2.46 290.71228375374494
6 2.87 748.8197057817309
7 3.28 1926.4214046822738
8 3.69 4953.466286799619
9 4.1 12733.89881249376
10 4.52 33494.08774450215
11 4.93 86098.31649831648
12 5.34 221303.3707865168
13 5.75 568805.6547845253
14 6.16 1462076.3832459182
15 6.57 3758184.0748921838
16 6.98 9658412.093081601
17 7.39 24798930.89354681
18 7.8 63803157.46421266
19 8.13 134697032.25806448
20 8.22 166081152.1486643
166081152.1486643


# history

In [ ]:
'Tnnt1': ['17051', '17302', '17545', '18454', '18455', '18462', '18463']

In [ ]:
image = load_image('../../assets/average_template_25_u8_xpad.v3draw')[:,:,:,20:-20]
template = np.zeros((list(image.shape[1:])+[4]))
template[:,:,:,0] = image[0]
template[:,:,:,1] = image[0]
template[:,:,:,2] = image[0]
template[:,:,:,3] = 255
template = template.astype(np.uint8)

fig,ax = plt.subplots(2,2,figsize=(60,20))
ax[0,0].imshow(template.max(axis=1))
ax[0,1].imshow(template.max(axis=0))
ax[1,0].imshow(template.max(axis=2))
print(template.shape)

plts = template.max(axis=1)
plts = template[:,160,:]
print(plts.shape,plts.dtype)
ax[1,1].imshow(plts)

In [ ]:
def rm_band(imgfile,regi):
    image = load_image(imgfile)[0]
    image = image.astype(np.uint8)
    if regi==1:
        template = load_image('../../assets/average_template_25_u8_xpad.v3draw')[0][:,:,20:-20]
    elif regi==0:
        template = load_image(imgfile.replace('.v3draw','_template.v3draw'))[0]
    elif regi==1000:
        return image
        template = load_image(imgfile.replace('18462','18463').replace('.v3draw','_template.v3draw'))[0]
    image[template==0] = 0
    return image

In [ ]:
fig,ax = plt.subplots(2,2,figsize=(60,30))
for imgfile in ['../brain/18454.v3draw','../brain/18454_local_registered_image.v3draw']:
    if 'local' in imgfile: r = 1
    else: r = 0
        
    image = rm_band(imgfile,r)
    image = image[:,int(image.shape[1]/2),:]#20:-20]
    print(image.shape)
    ax[r,0].imshow(image)
    if r==1:
        index = (image-65)<25
    else:
        image = rotate2d(image)
        index = (image-130)<50

    array = np.zeros((list(image.shape)+[4]))
    array[:,:,0] = image
    array[:,:,1] = image
    array[:,:,2] = image
    array[:,:,3] = 255
    array = array.astype(np.uint8)
    array[index,0] = 0
    array[index,2] = 0
    ax[r,1].imshow(array)    
    if r==1:
        plts[index,0] = 0
        plts[index,1] = image[index]
        plts[index,2] = 0
        imageio.imsave(imgfile.replace('.v3draw','.png'),plts)   
    else:
        imageio.imsave(imgfile.replace('.v3draw','.png'),array)
        

In [ ]:
fig,ax = plt.subplots(2,2,figsize=(60,30))
for imgfile in ['../brain/18455.v3draw','../brain/18455_local_registered_image.v3draw']:
    if 'local' in imgfile: r = 1
    else: r = 0
        
    image = rm_band(imgfile,r)
    image = image[:,int(image.shape[1]/2),:]#20:-20]
    print(image.shape)
    ax[r,0].imshow(image)
    if r==1:
        index = (image-65)<25
    else:
        image = rotate2d(image)
        index = (image-100)<50

    array = np.zeros((list(image.shape)+[4]))
    array[:,:,0] = image
    array[:,:,1] = image
    array[:,:,2] = image
    array[:,:,3] = 255
    array = array.astype(np.uint8)
    array[index,0] = 0
    array[index,1] = 0
    ax[r,1].imshow(array)
    if r==1:
        plts[index,0] = 0
        plts[index,1] = 0
        plts[index,2] = image[index]
        imageio.imsave(imgfile.replace('.v3draw','.png'),plts)   
    else:
        imageio.imsave(imgfile.replace('.v3draw','.png'),array)

In [ ]:
fig,ax = plt.subplots(2,2,figsize=(60,30))
for imgfile in ['../brain/18463.v3draw','../brain/18463_local_registered_image.v3draw']:
    if 'local' in imgfile: r = 1
    else: r = 0
        
    image = rm_band(imgfile,r)
    image = image[:,int(image.shape[1]/2),:]#20:-20]
    print(image.shape)
    ax[r,0].imshow(image)
#     if r==1:
#         index = (image-100)<50
#     else:
#         image = rotate2d(image)
#         index = (image-130)<50
    if r==1:
        index = (image-155)<=100
    else:
        image = rotate2d(image)
        index = (image-155)<=100
        
    array = np.zeros((list(image.shape)+[4]))
    array[:,:,0] = image
    array[:,:,1] = image
    array[:,:,2] = image
    array[:,:,3] = 255
    array = array.astype(np.uint8)
    array[index,1] = 0
    array[index,2] = 0
    ax[r,1].imshow(array)
    if r==1:
        plts[index,0] = image[index]
        plts[index,1] = 0
        plts[index,2] = 0
        imageio.imsave(imgfile.replace('.v3draw','.png'),plts)   
    else:
        imageio.imsave(imgfile.replace('.v3draw','.png'),array)

In [ ]:
plt.imshow(plts)
plt.savefig('brain/Tnnt.png',dpi=300)

In [ ]:
fig,ax = plt.subplots(2,2,figsize=(60,30))
for imgfile in ['../brain/18462.v3draw','../brain/18462_local_registered_image.v3draw']:
    if 'local' in imgfile: r = 1
    else: r = 1000
        
    image = rm_band(imgfile,r)
    image = image[:,int(image.shape[1]/2),:]#20:-20]
    print(image.shape)
    ax[r%2,0].imshow(image)
    if r==1:
        index = (image-100)<50
    else:
        image = rotate2d(image)
        index = (image-170)<90

    array = np.zeros((list(image.shape)+[4]))
    array[:,:,0] = image
    array[:,:,1] = image
    array[:,:,2] = image
    array[:,:,3] = 255
    array = array.astype(np.uint8)
    array[index,1] = 0
    ax[r%2,1].imshow(array)
    if r==1:
        plts[index,0] = image[index]
        plts[index,1] = 0
        plts[index,2] = image[index]
        imageio.imsave(imgfile.replace('.v3draw','.png'),plts)   
    else:
        imageio.imsave(imgfile.replace('.v3draw','.png'),array)

In [ ]:
((plts[index,1]+array[index,1])/2).shape
array[index,1].shape
np.array([(plts[index,1]+array[index,1])/2,array[index,1]]).max(axis=0).shape